In [14]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from random import randint
from os import urandom

In [15]:
class EncryptionOracleInterface:
    def encrypt(self, data: bytes) -> bytes:
        pass

In [16]:
class encryptionOracle(EncryptionOracleInterface):
    def __init__(self):
        self.history = []
        self.key = urandom(16)

    def encrypt(self, data: bytes) -> bytes:
        pt = urandom(randint(5, 10)) + data + urandom(randint(5, 10))

        if randint(0, 1):
            self.history.append("CBC")
            cipher = AES.new(self.key, AES.MODE_CBC)
        else:
            self.history.append("ECB")
            cipher = AES.new(self.key, AES.MODE_ECB)

        return cipher.encrypt(pad(pt, AES.block_size))

In [17]:
def ECB_CBC_detector(oracle: type[EncryptionOracleInterface]) -> str:
    data = b"A" * 48
    ct = oracle.encrypt(data)

    blocks = [ct[i : i + AES.block_size] for i in range(0, len(ct), AES.block_size)]

    return "ECB" if len(blocks) != len(set(blocks)) else "CBC"

In [20]:
oracle = encryptionOracle()
detections = []

for _ in range(10):
    detections.append(ECB_CBC_detector(oracle))

print(f"correct? {detections == oracle.history}\nactual: {oracle.history}\nexpect: {detections}")


correct? True
actual: ['ECB', 'ECB', 'ECB', 'CBC', 'ECB', 'CBC', 'CBC', 'CBC', 'ECB', 'ECB']
expect: ['ECB', 'ECB', 'ECB', 'CBC', 'ECB', 'CBC', 'CBC', 'CBC', 'ECB', 'ECB']
